## メモ

回帰分析の評価指標

* https://stats.biopapyrus.jp/glm/lm-evaluation.html

ユークリッド距離 vs コサイン類似度

* https://enjoyworks.jp/tech-blog/2242

ライブラリ使った方が楽そう

* https://github.com/NicolasHug/Surprise

レコメンドサンプル

 * https://github.com/NicolasHug/Surprise/blob/master/examples/top_n_recommendations.py

Python surprise で作る らくらく「レコメンドエンジン」（その１）

* https://www.salesanalytics.co.jp/datascience/datascience180/

MF系アルゴリズムのまとめ

* https://qiita.com/kondo-k/items/7a67881520da23229ce5

In [ ]:
! pip install surprise

In [ ]:
from surprise import NMF, SVD, SVDpp, KNNBasic, SlopeOne, Dataset, accuracy, Reader
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV
from google.colab import drive
from collections import defaultdict
import numpy as np
import pandas as pd
import random

In [ ]:
drive.mount('/content/drive')

In [ ]:
scores_tmp_df = pd.read_csv('drive/My Drive/dev/20230424_recommend_erogame/userbase_matrix.csv', encoding='utf-8')
user_df = pd.read_csv('drive/My Drive/dev/20230424_recommend_erogame/userbase_user_map.csv', encoding='utf-8')
game_df = pd.read_csv('drive/My Drive/dev/20230424_recommend_erogame/userbase_game_map.csv', encoding='utf-8')
# scores_df = pd.read_csv('drive/My Drive/dev/20230424_recommend_erogame/score_df_pickupuser_202310.csv' ,encoding='utf-8')
# scores_df = pd.read_csv('drive/My Drive/dev/20230424_recommend_erogame/score_df_pickupuser_202310v2.csv' ,encoding='utf-8')
scores_df = scores_tmp_df

In [ ]:
print("ユーザー数：" + str(len(user_df)))
print("ゲーム数：" + str(len(game_df)))
print("ユーザー数（抽出済み対象）：" + str(len(scores_df['uid'].unique())))

In [ ]:
scores_tmp_df.groupby('game_id')['score'].agg(['mean','count']).sort_values('count', ascending=False)

In [ ]:
## 評価数を全ユーザー合わせる
def random_pickupitem(df, max_item):
  dic = {}
  result_df = df.copy()
  for index, tmp in df.iterrows():
    if tmp.uid not in dic:
      dic[tmp.uid] = []
    dic[tmp.uid].append(index)
  for key in dic:
    delete_num = len(dic[key]) - max_item
    if delete_num == 0:
      continue
    delete_keys = random.sample(dic[key], delete_num)
    result_df.drop(index = delete_keys,inplace=True)
  return result_df
scores_df_pickupitem = random_pickupitem(scores_tmp_df,20)
# 時間かかるのでファイル化
# scores_df_pickupitem.to_csv('drive/My Drive/dev/20230424_recommend_erogame/score_df_pickupitem_202310.csv', index=False)
scores_df_pickupitem.to_csv('drive/My Drive/dev/20230424_recommend_erogame/score_df_pickupitem_202310v2.csv', index=False)

In [ ]:
## ユーザーを抽出
def random_pickupuser(df, max_user):
  del_list = []
  result_df = df.copy()
  delete_num = len(df['uid'].unique()) - max_user
  del_list = random.sample(df['uid'].unique().tolist(), delete_num)
  for key in del_list:
    delete_keys = df[df['uid']==key].index
    result_df.drop(index = delete_keys,inplace=True)
  return result_df
scores_df = random_pickupuser(scores_df_pickupitem,2000)
# 時間かかるのでファイル化
# scores_df.to_csv('drive/My Drive/dev/20230424_recommend_erogame/score_df_pickupuser_202310.csv', index=False)
scores_df.to_csv('drive/My Drive/dev/20230424_recommend_erogame/score_df_pickupuser_202310v2.csv', index=False)

In [ ]:
scores_df.groupby('game_id')['score'].agg(['mean','count']).sort_values('count', ascending=False)

In [ ]:
print(scores_df[scores_df['uid']==8].index)
# len(scores_df['uid'].unique())
# len(scores_df['game_id'].unique())
# random.sample(scores_df['uid'].unique().tolist(),1)

In [ ]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    count = 0
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
def get_info(list, target_user_index):
  print("user_id : " + user_df.iloc[target_user_index].uid + " - https://erogamescape.dyndns.org/~ap2/ero/toukei_kaiseki/user_infomation.php?user=" + user_df.iloc[target_user_index].uid)
  for tmp in list[target_user_index]:
    if tmp[0] in scores_tmp_df[scores_tmp_df['uid']==target_user_index].game_id.values:
      continue
    print(str(game_df.iloc[tmp[0]].game_id) + "," + str(tmp[1]))
    # print("game_id : " + str(tmp[0]) + "-" + str(game_df.iloc[tmp[0]].game_id) + " - " + str(tmp[1]) + " - https://erogamescape.dyndns.org/~ap2/ero/toukei_kaiseki/game.php?game=" + str(game_df.iloc[tmp[0]].game_id))

In [ ]:
# reader = Reader(rating_scale=(1,5))
reader = Reader(rating_scale=(1,10))
data = Dataset.load_from_df(scores_df[["uid", "game_id", "score"]],reader)

In [ ]:
# パラメータの調整
# SVD
# param_grid = {"n_factors":[20], "n_epochs": [20, 25, 30], "lr_all": [0.002, 0.005 ,0.01, 0.03]}
# gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)
#0.6648357155315309
#{'n_factors': 20, 'n_epochs': 25, 'lr_all': 0.005}
#1.3307434581038435
#{'n_factors': 20, 'n_epochs': 30, 'lr_all': 0.005}

# SVDpp
#param_grid = {"n_factors":[20, 50, 100], "n_epochs": [20, 25, 30], "lr_all": [0.002, 0.005 ,0.01, 0.1]}
#gs = GridSearchCV(SVDpp, param_grid, measures=["rmse", "mae"], cv=3)
#0.6627003262915983
#{'n_factors': 20, 'n_epochs': 25, 'lr_all': 0.005}

# NMF
param_grid = {"n_factors":[20, 25, 30], "n_epochs": [50, 100], "biased": [True, False]}
gs = GridSearchCV(NMF, param_grid, measures=["rmse", "mae"], cv=3)
#0.6659059049908401
#{'n_factors': 20, 'n_epochs': 50, 'biased': True}
#1.3366060504042678
#{'n_factors': 25, 'n_epochs': 50, 'biased': True}

# KNNBasic
#param_grid = {"k":[20, 40, 60, 100], "min_k":[1, 5, 10], "sim_options": {'name':['pearson_baseline','cosine','msd','pearson']}}
#gs = GridSearchCV(KNNBasic, param_grid, measures=["rmse", "mae"], cv=3)
# 0.7456638991491215
# {'k': 40, 'min_k': 5, 'sim_options': {'name': 'msd', 'user_based': True}}

# SlopeOne
#param_grid = {}
#gs = GridSearchCV(SlopeOne, param_grid , measures=["rmse", "mae"], cv=3)
#0.7897393384714445
#{}

gs.fit(data)

# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

In [ ]:
# algo=SVDpp(n_factors=20,n_epochs=25,lr_all=0.005)
# algo=NMF(n_factors=20,n_epochs=50,biased=True)
algo=NMF(n_factors=25,n_epochs=50,biased=True)
# algo=SVD(n_factors=20,n_epochs=20,lr_all=0.01)
# algo=SVD(n_factors=20,n_epochs=30,lr_all=0.005)
# algo = KNNBasic(k=40, min_k=5, sim_options={'name':'pearson_baseline'})

In [ ]:
cross_validate(algo, data, measures=['RMSE','MAE'], cv=5, verbose=True)

In [ ]:
trainset = data.build_full_trainset()
testset = trainset.build_anti_testset()
algo.fit(trainset)
predictions = algo.test(testset)

In [ ]:
predictions

In [ ]:
top_10 = get_top_n(predictions, 100)

In [ ]:
scores_tmp_df[scores_tmp_df['uid']==2].game_id.values

In [ ]:
get_info(top_10, 2)

In [ ]:
scores_df.groupby('game_id')['score'].agg(['mean','count']).sort_values('count', ascending=False)